In [21]:
#Project = Analyzing SNP Data 

#Analyzing data from sets of single nucleotide polymorphisms (SNPs) that 
#commonly vary in the human population. There are two datasets, extracted from 
#http://23andme.com, one from the fictitious male, Greg Mendel, and the other from his wife, Lilly 
#Mendel.  

In [2]:
#Data in these files are poorly formatted. For example the a line in the file:
# rs3094315chr1-742429(A,G) has information that can be parsed out into:
#id Chr Position SNP1 SNP2 
#rs3094315 1 742429 A G 

#Therefore, the next step will extract all useful data from the files

In [10]:
greg_file = open('GregMendel_SNPs.txt','r')
lilly_file = open('LillyMendel_SNPs.txt','r')

print('Preview first line of Greg_file')
print(greg_file.readlines()[0:1])

print('Preview first line of Lily_file')
print(lilly_file.readlines()[0:1])

greg_file.close()
lilly_file.close()

Preview first line of Greg_file
['rs3094315chr1-742429(A,G)\n']
Preview first line of Lily_file
['rs3094315chr1-742429(A,A)\n']


In [7]:
def read_SNP_file(filename): 
    '''
    Purpose: Create a dictionary parsing out important information from filename provided
    Attributes: filename.txt of the SNPs stored
    Return: Dictionary with parsed out columns - ID, SNP, Chromose, Position
    '''
    id_list = []
    chrom = []
    pos = []
    SNP = []
    file = open(str(filename),'r')
    for line in file.readlines():
        line = line.strip()
        line.rstrip()
        id_list.append(line.split('chr')[0])
        SNP1 = line.split('chr')[1].split('(')[1].split(',')[0]
        SNP2 = line.split('chr')[1].split('(')[1].split(',')[1].split(')')[0]
        SNP.append(SNP1+SNP2)
        chrom.append((line.split('chr')[1]).split('-')[0])
        pos.append(((line.split('chr')[1]).split('-')[1]).split('(')[0])
    dict_SNP = {'ID':id_list,'SNP': SNP,'chr':chrom, 'position':pos}
    return dict_SNP

In [11]:
Greg_dict = read_SNP_file('GregMendel_SNPs.txt')
Lilly_dict = read_SNP_file('LillyMendel_SNPs.txt')

In [16]:
#On Chromosome 10, finding the largest region of shared SNPs between Lilly and Greg. 
#Below is an example of a region of shared SNPs between positions 31,123 and 31625.
'''
Chromosome Position Lilly Greg 
10 31,000 AA AT 
10 31,123 TT TT 
10 31,319 AT AT 
10 31,625 CC CC 
10 31,779 GA CC
'''

'\nChromosome Position Lilly Greg \n10 31,000 AA AT \n10 31,123 TT TT \n10 31,319 AT AT \n10 31,625 CC CC \n10 31,779 GA CC\n'

In [20]:
global_start = 0
global_end = 0
global_max =  0
local_start = 0
local_end = 0
local_max = 0
for index in range(len(Greg_dict['ID'])):
    mychr = Greg_dict['chr'][index]
    if mychr == '10':
        mypos = Greg_dict['position'][index]
        Greg_SNP = Greg_dict['SNP'][index]
        Lilly_SNP = Lilly_dict['SNP'][index]
        if Lilly_SNP == Greg_SNP: 
            if local_max == 0:
                local_start = Greg_dict['position'][index] 
                local_max += 1
            else:
                local_max += 1
        else:
            local_end = Greg_dict['position'][index - 1]
            if local_max > global_max: #Current shared region has more SNPs than the maximum region so far
                global_max = local_max
                global_start = local_start
                global_end = local_end
            #reset/reinitialize local variables
            local_max = 0
            local_start = 0
            local_end = 0

print('max shared SNPs region:',global_max)
print('Largest SNPs shared region between Lilly and Greg on Chromosome 10 was between position {} and position {}'.format(global_start,global_end))

max shared SNPs region: 114
Largest SNPs shared region between Lilly and Greg on Chromosome 10 was between position 73515890 and position 74789905


In [28]:
'''
SNP_definitions.txt contains descriptions/information about the effects of various SNPs
Therefore, next step is to extract the data and create a dictionary
Key = (SNP ID, SNP); Value = Description of 
'''

'\nSNP_definitions.txt contains descriptions/information about the effects of various SNPs\nTherefore, next step is to extract the data and create a dictionary\nKey = (SNP ID, SNP); Value = Description of the effect\n'

In [29]:
SNP_id = []
descrip = []
genotype = []
SNPdef_file = open('SNP_definitions.txt','r')

for line in SNPdef_file.readlines()[1:]:
    line = line.strip()
    line.rstrip()
    line = line.split('\t')
    SNP_id.append(line[0])
    genotype.append(line[1])
    descrip.append(line[2])
SNPdef_dict = {}
for i in range(len(descrip)):
    mykey = SNP_id[i], genotype[i]
    SNPdef_dict[mykey] = descrip[i]
        
SNPdef_file.close()

In [30]:
#How can this dictionary be used? To answer questions like:
#"what does the region between positions 
#22070000 and 22106000 on chromosome 9 suggest about Greg’s chance of heart disease?
#What about Lilly’s chance of heart disease? "

In [33]:
def disease(dictionary, pos1, pos2, chromosome):
    disease_list = []
    for i in range(len(dictionary['chr'])):
        if (dictionary['chr'][i] == str(chromosome)) and (pos1 < int(dictionary['position'][i]) < pos2):
            disease_SNP = dictionary['SNP'][i]
            disease_ID = dictionary['ID'][i]
            for keys in SNPdef_dict.keys():
                if (disease_ID,disease_SNP) == keys:
                    disease_list.append(SNPdef_dict[keys])
    return print(*disease_list,sep = '\n')

print('Gregs risk for heart attack')
disease(Greg_dict, 22070000, 22106000, 9)
print('-----------------------------')
print('Lillys risk for heart attack')
disease(Lilly_dict, 22070000, 22106000, 9)

Gregs risk for heart attack
"1.24x increased myocardial infarction risk, 1.29x increased aneurysm risk"
~1.2x increased risk for heart disease
1.4x increased risk for heart disease
increased risk for heart disease
-----------------------------
Lillys risk for heart attack
normal
normal
normal
normal
